# 2. Programmierprojekt: Local Search

## $n$-Damen Problem

Wir modellieren das $n$-Damen Problem wie folgt: Jeder Zustand im Suchraum ist eine Permutation des Vektors $(0, \dotsc, n-1)$. Damit sind die Aktionen die von einem Zustand möglich sind Vertauschungen von Zahlen.

In [1]:
import random

def generate_random_conf(n: int) -> list:
  # Generate random permutation of (0,...,n-1)
  conf = [i for i in range(n)]
  random.shuffle(conf)
  return conf

def swap(conf, i, j):
  # Swap positions i,j inplace
  conf[i], conf[j] = conf[j], conf[i]

In [ ]:
conf = generate_random_conf(8)
swap(conf, 3, 5)
conf

Implentieren Sie die folgende Funktion `conflicts`, welche für jede Dame die Anzahl der Bedrohungen aufsummiert. Überlegen Sie, welche Art von Bedrohungen durch die Modellierung überhaupt möglich sind.

In [ ]:
def conflicts(queens) -> int:
  """ Heuristic that indicates number of beatable queens on the right"""
  
  conflicts = 0
  for i in range(len(queens)):
    # For better initialization
    if queens[i] == float("-inf"):
        continue
    for j in range(i + 1, len(queens)):
        conflicts += int(queens[i] == queens[j] or queens[i] + j - i == queens[j] or queens[i] - j + i == queens[j])
  return conflicts

In [ ]:
conf1 = [0, 1, 2, 3, 4, 5, 6, 7] # conflicts = 28
conf2 = [17, 22, 11, 5, 2, 6, 12, 15, 21, 19, 10, 8, 0, 3, 1, 20, 23, 9, 14, 18, 13, 24, 16, 4, 7] # conflicts = 0
conf3 = [15, 12, 9, 16, 0, 18, 6, 11, 19, 7, 13, 3, 10, 4, 1, 2, 8, 5, 14, 17] # conflicts = 12


assert conflicts(conf1) == 28
assert conflicts(conf2) == 0
assert conflicts(conf3) == 12

Nutzen sie die obige Funktion als Heuristik für den A*-Algorithmus. Testen sie bis zu welchem $n$ der Algorithmus eine Lösung in unter zwei Minuten findet. Starten Sie dabei immer von einer zufälligen Startkonfiguration.

In [ ]:
from queue import PriorityQueue


def queens_a_star(n: int) -> list[int]:
    """ Find solution with A*; not feasible for n >= 50 """
    
    conf = generate_random_conf(n)
    
    queue = PriorityQueue()
    # stores f, h, g, configuration
    queue.put((conflicts(conf), conflicts(conf), 0, conf))
    visited = []
    
    while not queue.empty():
        entry = queue.get()
        g_score, state = entry[2], entry[3]
        visited.append(state)

        for successor in successors_astar(state):
            if not conflicts(successor):
                return successor
            if successor in visited:
                continue
                
            new_g_score = g_score + 1
            h = conflicts(successor)
            f_score = h + new_g_score
            queue.put((f_score, h, new_g_score, successor))
            visited.append(successor)

    return [-1]


def successors_astar(queens) -> list[list[int]]:
    """ Returns list of successor states; action is swapping """
    successors = []
    for i in range(len(queens)):
        for j in range(len(queens)):
            if i == j:
                continue
            copy = queens.copy()
            swap(copy, i, j)
            successors.append(copy)
    return successors
    

Wir wollen im Folgenden einen Local-Search Ansatz nutzen, um das Problem zu lösen. Implementieren Sie nun den Hill-Climbing Algorithmus aus der Vorlesung. Der Algorithmus sollte zusätzlich maximal $k$ Seitwärtszüge erlauben.

In [ ]:
def queens_hill_climb(n, k=0) -> tuple[list[int], int]:
    return hill_climb_core(generate_random_conf(n), k)
  
def hill_climb_core(conf, k=0) -> tuple[list[int], int]:
    """ Searches solution by hill-climbing; k: possible side steps """  
    conflicts_amount = conflicts(conf)
    improvement, side_steps = 1, 0
    visited = []
  
    while side_steps <= k  and conflicts_amount != 0:
        visited.append(conf)
        new_conf, new_conflict_amount = successors(conf, visited)
      
        improvement = conflicts_amount - new_conflict_amount
      
        if improvement:
            side_steps = 0
        else:
            side_steps += 1
          
        conf, conflicts_amount = new_conf, new_conflict_amount
      
    return conf, conflicts(conf)


def successors(queens, visited) -> list[list[int]]:
    """ Returns best successor"""
    best_successor = (queens, conflicts(queens))
    for i in range(len(queens)):
        for j in range(len(queens)):
            if i == j:
                continue
            copy = queens.copy()
            swap(copy, i, j)
            best_successor = (copy, conflicts(copy)) if (conflicts(copy) <= best_successor[1] and copy not in visited) else best_successor
    return best_successor


Evaluieren Sie den Algorithmus empirisch. Testen sie für verschiedene $n$ und $k$, wie groß die Erfolgsrate des Algorithmus ist. Überlegen Sie sich eine geeignete Visualisierung ihrer Ergebnisse.

In [ ]:
n_samples = 200
k = [0, 5, 10, 20, 50, 100]

total_amount, total_hits = 0, 0
for i in k:
    amount, hits = 0, 0
    for n in range(8, n_samples):
        amount += 1
        hits += int(not queens_hill_climb(n, i)[1])
    print("Success rate for k= " + str(i) + " : " + str((hits / amount) * 100) + "%")
    total_amount += amount
    total_hits += hits
print("Overall success rate: " + str((total_hits / total_amount) * 100) + "%")

Implementieren sie nun Hill-Climbing mit Random-Restarts, um immer optimale Lösungen zu finden. Berechnen Sie auch, wie viele Restarts nötig waren, um eine optimale Lösung zu finden. Bis zu welchem $n$ können sie mit diesem Ansatz in weniger als zwei Minuten Lösungen finden? (Probieren sie auch verschiedene $k$)

In [ ]:
def queens_random_restart(n, k=0) -> tuple[list[int], int]:
    """ Restart hill climbing till solution is found; not feasible for n >= 65   """
    restart = 0
    conf, conflict_amount = queens_hill_climb(n, k)
    while conflict_amount:
        restart += 1
        conf, conflict_amount = queens_hill_climb(n, k)
    return conf, restart

Eine weitere Verbesserung kann erreicht werden, in dem die initiale Konfiguration nicht rein zufällig gewählt wird. Es kann versucht werden, anfangs eine Konfiguration zu finden, in der möglichst viele Damen bereits konfliktfrei gesetzt sind. Die restlichen Damen sollten dann mit möglichst wenigen Konflikten gesetzt werden.

Implementieren Sie darauf basierend einen verbesserten Generator für die Startkonfiguration und testen Sie, ob damit noch größere Probleminstanzen gelöst werden können.

In [ ]:
from random import randrange
import numpy as np


def better_initial_configuration(n: int, k=0) -> tuple[list[int], int]:
    """ Initialize conf with minimal conflicts """
    conf = [float("-inf")] * n
    f = np.zero(n, n)
    possible_queens = [i for i in range(n)]
    conf[0] = possible_queens.pop(randrange(n))
    f = magic(f, conf[0], 0)
    
    for i in range(1, n):
        copy = conf.copy()
        min_conflicts = possible_queens[0], 0 # index, conflicts_amount

        # TODO min overlap
        
        for j in possible_queens:
            copy[i] = j
            if conflicts(copy) <= min_conflicts[1]:
                min_conflicts = j, conflicts(copy)

        
        conf[i] = min_conflicts[0]
        possible_queens.remove(conf[i])
    print(conf, conflicts(conf))
    return hill_climb_core(conf, k)

def magic(f, x, y):
    f[:, y] = -1
    f[x, :] = -1
    # TODO diagonal
    return f
    

print(better_initial_configuration(20))
print("\n")
print(queens_hill_climb(20))

In [ ]:
n_samples = 200
k = [0, 5, 10, 20, 50, 100]

total_amount, total_hits = 0, 0
for i in k:
    amount, hits = 0, 0
    for n in range(8, n_samples):
        amount += 1
        hits += int(not better_initial_configuration(n, i)[1])
    print("Success rate for k= " + str(i) + " : " + str((hits / amount) * 100) +"%")
    total_amount += amount
    total_hits += hits
print("Overall success rate: " + str((total_hits / total_amount) * 100) +"%")



---



## Travelling-Salesman Problem

In diesem Teil soll das TSP mithilfe von Local Search approximiert werden. Für diese Aufgabe betrachten wir ausschließlich das symmetrische TSP, bei dem die Kanten der Graphen ungerichtet sind. Es gibt folglich für einen Graph nur eine Tour.

Wir verwenden zur Darstellung der Graphen das Paket `networkx` (https://networkx.org/documentation/latest/). Außerdem das Paket `tsplib95` (https://tsplib95.readthedocs.io/en/stable/index.html) um die Algorithmen mit Benchmarks zu testen.

In [2]:
import networkx as nx
import tsplib95
import os
import random
import math

Implementieren Sie die Funktion `import_benchmarks`, welche die verschiedenen TSP Instanzen zusammen mit den Lösungen aus dem Ordner `tsp_benchmarks` importiert und eine Liste aus Tupeln der Form `(G, optimal_solution)` zurückgibt, bestehend aus dem Graphen als `networkx`-Objekt und dem Gewicht der optimalen Lösung.

In [ ]:
def import_benchmarks(path='./tsp_benchmarks/') -> list[tuple[nx.Graph, int]]:
    """ Loads TSP problems with optimal solution """
    res = []
    dir_list = sorted(os.listdir(path))
    for i in range(len(dir_list) - 1):
        # make sure tsp and tour belong to each other
        if dir_list[i].split('.')[0] != dir_list[i+1].split('.')[0]:
            continue

        opt = tsplib95.load(path + dir_list[i])
        problem = tsplib95.load(path + dir_list[i+1])
        G = problem.get_graph()
        print(dir_list[i+1], dir_list[i])
        
        try:
            optimal_solution = problem.trace_tours(opt.tours)[0]
        except IndexError:
            optimal_solution = None
            
        res.append((G, optimal_solution))
        
    return res
    
a = import_benchmarks()
print(len(a))

a280.tsp a280.opt.tour
att48.tsp att48.opt.tour
bayg29.tsp bayg29.opt.tour
bays29.tsp bays29.opt.tour
berlin52.tsp berlin52.opt.tour
brg180.tsp brg180.opt.tour
ch130.tsp ch130.opt.tour
ch150.tsp ch150.opt.tour
eil101.tsp eil101.opt.tour
eil51.tsp eil51.opt.tour
eil76.tsp eil76.opt.tour
fri26.tsp fri26.opt.tour
gr120.tsp gr120.opt.tour
gr202.tsp gr202.opt.tour
gr24.tsp gr24.opt.tour
gr48.tsp gr48.opt.tour
gr666.tsp gr666.opt.tour
gr96.tsp gr96.opt.tour
kroA100.tsp kroA100.opt.tour
kroC100.tsp kroC100.opt.tour
kroD100.tsp kroD100.opt.tour
lin105.tsp lin105.opt.tour
pa561.tsp pa561.opt.tour
pcb442.tsp pcb442.opt.tour
pr1002.tsp pr1002.opt.tour
pr2392.tsp pr2392.opt.tour
pr76.tsp pr76.opt.tour
rd100.tsp rd100.opt.tour
st70.tsp st70.opt.tour


Implementieren Sie nun die folgenden drei Local-Search Algorithmen um das TSP zu lösen. Genau wie beim $n$-Damen Problem, kann auch beim TSP eine Lösung als Permutation der Knoten des Graphen gesehen werden. Für das Hill-Climbing und dem Simulated Annealing, besteht eine Aktion daraus, zwei Knoten auf dem Rundweg zu vertauschen. Beim EX3-Algorithmus ist die Aktion unten beschrieben.

**1. Simple Hill-Climbing**

In [ ]:
def tsp_hill_climb(G: nx.Graph, succ=successors_hc) -> int:
    """ Simple Hill Climbing """
    conf = list(G.nodes)
    tour_cost = calculate_tour_cost(G, conf)
    visited = []
    improvement = 1
    while improvement :
        visited.append(conf)
        new_conf, new_tour_cost = succ(G, conf, visited)
        improvement = tour_cost - new_tour_cost
          
        conf, tour_cost = new_conf, new_tour_cost
    return conf, calculate_tour_cost(G, conf)

def calculate_tour_cost(G, t) -> int:
    """ I """
    total_cost = 0
    tour = t + [t[0]]
    for i in range(len(tour) - 1):
        u, v = tour[i], tour[i + 1]
        # Add the weight of the edge (u, v)
        total_cost += G[u][v]['weight']
    return total_cost

def successors_hc(G, conf, visited) -> list[list[int]]:
    """ Returns best successor """
    best_successor = (conf, calculate_tour_cost(G, conf))
    for i in range(len(conf)):
        for j in range(len(conf)):
            if i == j:
                continue
            copy = conf.copy()
            swap(copy, i, j)
            best_successor = (copy, calculate_tour_cost(G,copy)) if (calculate_tour_cost(G, copy) <= best_successor[1] and copy not in visited) else best_successor
    return best_successor

print(tsp_hill_climb(a[2][0]))

**3. Simulated Annealing**

In [ ]:
def tsp_simulated_annealing(G: nx.Graph, temperature: float) -> int:
    """ Takes worse steps dependend on temperature """
    conf = list(G.nodes)
    tour_cost = calculate_tour_cost(G, conf)
    visited = []
    
    for t in range(temperature, -1, -1):
        if t ==  0:
            return conf, calculate_tour_cost(G, conf)
            
        visited.append(conf)
        sucessor = conf.copy()
        random.shuffle(sucessor)
        #while sucessor in visited: # dangeer infinity
            #sucessor = random.shuffle(conf)
        new_tour_cost = calculate_tour_cost(G, sucessor)
        
        if new_tour_cost < tour_cost:
            conf = sucessor
        else:
            delta = abs(tour_cost - new_tour_cost)
            p = math.exp(delta / temperature)
            if random.random() < p:
                conf = sucessor
                
# TODO good temp    
print(tsp_simulated_annealing(a[2][0], 100))

**2. EX3-Algorithmus**

Der EX3-Algorithmus funktioniert ähnlich wie Hill-Climbing, benutzt allerdings eine etwas andere Nachfolgerfunktion. Die Nachfolger eines Rundweges werden bestimmt, indem zunächst zufällig **drei** Kanten aus dem bisherigen Rundweg ausgewählt werden. Dies zerlegt den Rundweg in drei Teilrundwege. Für das erneute zusammenfügen gibt es danach 8 Möglichkeiten (wie im Bild unten zu sehen ist). Der Algorithmus testet nun für jede 3 Kanten alle Nachfolger und wählt denjenigen aus, der die Kosten am stärksten verringert. Dies wird so lange gemacht, bis sich der Rundweg nicht mehr verkürzt.

![EX3 Visualisation](three_opt.png "EX3")

In [ ]:


def tsp_ex3(G: nx.Graph) -> int:
    tsp_simulated_annealing(G, successors_ex3)

def successors_ex3(G, conf, visited) -> list[list[int]]:
    """ Returns best successor of ex3 """
    best_successor = (conf, calculate_tour_cost(G, conf))
    l = len(conf)
    x, y, z = random.randint(l), random.randint(l), random.randint(l)
    copy = conf.copy()
    b = 
    
    """
    for i in range(len(conf)):
        for j in range(len(conf)):
            if i == j:
                continue
            copy = conf.copy()
            swap(copy, i, j)
            best_successor = (copy, calculate_tour_cost(G,copy)) if (calculate_tour_cost(G, copy) <= best_successor[1] and copy not in visited) else best_successor
    """
    return best_successor
    

**Evaluierung**

Vergleichen Sie die drei Algorithmen miteinander. Nutzen Sie dafür die bereitgestellten Benchmarks und die Länge der Optimallösung. Sie sollten Ihre Ergebnisse geeignet visualisieren.

In [ ]:
bm = import_benchmarks()
average_deviation_hc, average_deviation_sa, average_deviation_hex = 0, 0, 0
for b in bm:
    if b[1] == None:
        continue
    hc, sa, ex, opt = tsp_hill_climb(b[0]), tsp_simulated_annealing(b[0]), tsp_ex3(b[0]), b[1]
    ad_hc, ad_sa, ad_hex += hc / opt - 1, sa / opt - 1, ex / opt - 1

l = len(bm)
# TODO *100
print("Average deviation from optimum    HC: " + str(ad_hc/l) + "%, SA: " + str(ad_hc/l)) + "%, EX3: " + str(ad_hc/l)) + "%") 